In [166]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from slugify import slugify
from nanoid import generate
from deep_translator import GoogleTranslator

Link_grabber

In [133]:
def remove_params(url: str) -> str:
    return url.split("?", 1)[0]


def epam():
    url = "https://www.epam.com/services/vacancy/search?locale=en&limit=200&recruitingUrl=%2Fcontent%2Fepam%2Fen%2Fcareers%2Fjob-listings%2Fjob&query=&country=Mexico&sort=relevance&offset=0&searchType=placeOfWorkFilter&_=1709566523032"
    response = requests.get(url)
    json_data = response.json()
    vacancy_links = []
    for item in json_data['result']:
        vacancy_links.append(item['url'])
    return vacancy_links

In [134]:
links = epam()

In [136]:

def get_job_details_epam(url):
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")

        title_element = soup.find(class_="vacancy-details-23__job-title")
        title = title_element.text if title_element else None

        location_element = soup.select_one(
            'span[class="vacancy-details-23__location"]'
        )
        location = location_element.get_text(strip=True) if location_element else None

        summary_element = soup.find("div", class_="vacancy-details-23__top-description")
        summary = summary_element.text if summary_element else None

        content_by_title = {}
        div_contenido = soup.find('div', class_='vacancy-details-23__content-holder')
        if div_contenido:
            for h3_tag in div_contenido.find_all('h3'):
                elementos_li = []
                ul_tag = h3_tag.find_next_sibling('ul')
                for li_tag in ul_tag.find_all('li'):
                    elementos_li.append(li_tag.text.strip())
                content_by_title[h3_tag.text.strip()] = elementos_li
        
        datos = {
            "externalLink": links,
            "name": title,
            "country": location,
            "description": summary,
            **content_by_title
        }

        df = pd.DataFrame([datos])

        return df
    except Exception as e:
        print(f"Error: {str(e)}")
        return None
    


def apple(links: list[str]) -> pd.DataFrame:
    df_url = pd.DataFrame({"externalLink": links})
    resultados = []
    for url in df_url["externalLink"]:
        detalles_vacante = get_job_details_epam(url)
        if detalles_vacante is not None:
            resultados.append(detalles_vacante)
    df = pd.concat(resultados, ignore_index=True)
    return df

In [137]:
df = apple(links)

In [138]:
df

,externalLink,name,country,description,Responsibilities,Requirements,Nice to have,We Offer,Conditions,Technologies,Project technologies and tools,What You’ll Do,What You Have,About EPAM,Must have experience,We offer
0,[https://www.epam.com/careers/job-listings/job...,\n Application Support (Mobile)\n ...,"Guadalajara,\n \n ...",\n Are you a skilled Application S...,[Provide technical assistance and support as p...,[2-5 years of experience with mobile developme...,[Experience using Firebase],"[Career plan and real growth opportunities, Un...","[By applying to our role, you are agreeing tha...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[https://www.epam.com/careers/job-listings/job...,\n Big Data Support Engineer\n \...,"Guadalajara,\n \n ...",\n EPAM is a leading global provide...,[Provide support to customers using Cloud Plat...,[2+ years of experience involved with projects...,NaN,"[Career plan and real growth opportunities, Un...","[By applying to our role, you are agreeing tha...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[https://www.epam.com/careers/job-listings/job...,\n Cloud Operations (Linux)\n \n...,"Guadalajara,\n \n ...",\nDESCRIPTIONAre you a seasoned professional w...,[Provide support to customers using Cloud Plat...,[At least 2 years’ experience as a Systems Eng...,NaN,"[Career plan and real growth opportunities, Un...","[By applying to our role, you are agreeing tha...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[https://www.epam.com/careers/job-listings/job...,\n Data Consultant / Data Science Engin...,"Mexico City,\n \n ...",\nDESCRIPTIONWe are seeking a talented Data C...,"[Perform advanced data analysis and modeling, ...","[Proficiency in Python, R, SQL, and data visua...",[Experience with industry-related data analyti...,"[Career plan and real growth opportunities, Un...","[By applying to our role, you are agreeing tha...","[Python, R, SQL]",NaN,NaN,NaN,NaN,NaN,NaN
4,[https://www.epam.com/careers/job-listings/job...,\n Data Integration Engineer\n \...,"Mexico City,\n \n ...",\nDESCRIPTIONJoin our dynamic team as a Data ...,[Collaborate with Finance teams and Project Ma...,[Proficiency in SQL for data manipulation and ...,[Familiarity with Google-based internal tools ...,"[Career plan and real growth opportunities, Un...","[By applying to our role, you are agreeing tha...","[SQL, Data ETL, Data Integration, Data Warehou...",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,[https://www.epam.com/careers/job-listings/job...,\n Sr. Salesforce Developer\n \n...,Remote,\n EPAM is a leading global provide...,"[Software development, Following Our developme...",[•Excellent written and oral communication ski...,"[Salesforce Platform Developer 1, Knowledge of...","[Career plan and real growth opportunities, Un...","[By applying to our role, you are agreeing tha...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,[https://www.epam.com/careers/job-listings/job...,\n Sr. iOS Developer\n \n ...,Remote,\n EPAM is a leading global provide...,[iOS Swift development throughout the entire l...,"[5+ years of production experience, Ownership ...",NaN,"[Career plan and real growth opportunities, Un...","[By applying to our role, you are agreeing tha...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,[https://www.epam.com/careers/job-listings/job...,\n Testing/QA Manager - Microsoft Dynam...,Remote\n \n \n ...,\nDESCRIPTIONAre you a seasoned Testing/QA Ma...,[Develop comprehensive test strategy and appro...,[In-depth knowledge of Microsoft Dynamics 365 ...,"[Microsoft Dynamics 365: Finance and CRM, Auto...","[Career plan and real growth opportunities, Un...","[By applying to our role, you are agreeing tha...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,[https://www.epam.com/careers/job-listings/job...,\n UX Content Designer / UX Writer\n ...,Remote,\nDESCRIPTION Join Us as a UX Content Designer...,[Collaborate with our clients to transform bus...,[Minimum of 5 years of professional content st...,[Experience evaluating content against accessi...,"[Career plan and 

In [139]:
columnas_a_limpiar = ['name', 'country', 'description', 'Responsibilities', 'Requirements', 'Nice to have', 'We Offer','Technologies','Project technologies and tools','What You’ll Do','What You Have', 'About EPAM','Must have experience', 'We offer']

#Recorro con un for cada una de las columnas
for columna in columnas_a_limpiar:
    df[columna] = df[columna].replace('•', '', regex=False)
    df[columna] = df[columna].replace('\n', ' ', regex=False)
    df[columna] = df[columna].fillna('')
    df[columna] = df[columna].replace('[', '', regex=False)
    df[columna] = df[columna].replace(']', '', regex=False)


columnas_a_unir = ['Responsibilities', 'Requirements', 'Nice to have', 'We Offer','Technologies','Project technologies and tools','What You’ll Do','What You Have', 'About EPAM','Must have experience']
df['Req_txt'] = df[columnas_a_unir].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
df =df.drop(columnas_a_unir, axis=1)

df['country'] = df['country'].str.split(',').str[0]
df['country'] = df['country'].replace('Remote', '', regex=False)
df['country'] = df['country'].replace(' Remote', '', regex=False)
df['country'] = df['country'].replace('Mexico', '', regex=False)
df['country'] = df['country'].replace('Remote                                                                in México', '', regex=False)
df['country'] = df['country'].replace('Remote                                                                in México.', '', regex=False)
df['country'] = df['country'].replace('Remote                                                                in Mexico', '', regex=False)
df['country'] = df['country'].replace('Remote                                                                in Mexico.', '', regex=False)
df['country'] = df['country'].replace('Mexico                                                                                              in Ensenada & Guadalajara', 'Ensenada & Guadalajara', regex=False)
df = df.rename(columns={'country': 'city'})


In [140]:
df

,externalLink,name,city,description,Conditions,We offer,Req_txt
0,[https://www.epam.com/careers/job-listings/job...,\n Application Support (Mobile)\n ...,Guadalajara,\n Are you a skilled Application S...,"[By applying to our role, you are agreeing tha...",,['Provide technical assistance and support as ...
1,[https://www.epam.com/careers/job-listings/job...,\n Big Data Support Engineer\n \...,Guadalajara,\n EPAM is a leading global provide...,"[By applying to our role, you are agreeing tha...",,['Provide support to customers using Cloud Pla...
2,[https://www.epam.com/careers/job-listings/job...,\n Cloud Operations (Linux)\n \n...,Guadalajara,\nDESCRIPTIONAre you a seasoned professional w...,"[By applying to our role, you are agreeing tha...",,['Provide support to customers using Cloud Pla...
3,[https://www.epam.com/careers/job-listings/job...,\n Data Consultant / Data Science Engin...,Mexico City,\nDESCRIPTIONWe are seeking a talented Data C...,"[By applying to our role, you are agreeing tha...",,['Perform advanced data analysis and modeling'...
4,[https://www.epam.com/careers/job-listings/job...,\n Data Integration Engineer\n \...,Mexico City,\nDESCRIPTIONJoin our dynamic team as a Data ...,"[By applying to our role, you are agreeing tha...",,['Collaborate with Finance teams and Project M...
...,...,...,...,...,...,...,...
100,[https://www.epam.com/careers/job-listings/job...,\n Sr. Salesforce Developer\n \n...,,\n EPAM is a leading global provide...,"[By applying to our role, you are agreeing tha...",,"['Software development', 'Following Our develo..."
101,[https://www.epam.com/careers/job-listings/job...,\n Sr. iOS Developer\n \n ...,,\n EPAM is a leading global provide...,"[By applying to our role, you are agreeing tha...",,['iOS Swift development throughout the entire ...
102,[https://www.epam.com/careers/job-listings/job...,\n Testing/QA Manager - Microsoft Dynam...,Remote\n \n \n ...,\nDESCRIPTIONAre you a seasoned Testing/QA Ma...,"[By applying to our role, you are agreeing tha...",,['Develop comprehensive test strategy and appr...
103,[https://www.epam.com/careers/job-listings/job...,\n UX Content Designer / UX Writer\n ...,,\nDESCRIPTION Join Us as a UX Content Designer...,"[By applying to our role, you are agreeing tha...",,['Collaborate with our clients to transform bu...


In [141]:
skills = ['Flowcharts', 'Media Coverage', 'Digital Trafficker', 'Wireframes',
    'Infrastructure', 'Social Listening', 'Audiovisual Production', 'Microsoft SQL', 'Branding', 'Marketing',
     'Social Media', 'Personnel Management', 'Marketing Plan', 'Adobe Programs', 'Javascript Programming',
    'Content Manager', 'C# Programming', 'UX Design', 'Illustration', 'Data Analytics', 'Advertising Sales',
    'Project Management Office (PMO)', 'Adobe Illustrator', 'Jira', 'Windows Server', 'Insight Detection',
    'Sales Closing', 'Account Management', 'Team Leadership', 'Inbound Marketing', 'Freelance Artist', 'Figma', 'FX Makeup',
     'ReactJS', 'Social Ads', 'Dashboards', 'ERP (Enterprise Resource Planning) System', 'Creative Strategy',
    'Video Editing', 'Pressure Management and Goal Achievement', 'Communication Skills', 'Programming', 'Software Support',
     'Content Management', 'Journalism', 'Bioprocesses', 'Excel', 'Client Presentations', 'Storytelling', 'Benchmarks',
    'Node.js', 'Planning', 'Screen Printing', 'Digital Marketing', 'Google Cloud Platform', 'Customer Service', 'PHP',
    'Front-End Design', 'Design Thinking', 'Data Analysis', 'Client Prospecting', 'Copywriting', 'SQL Database', 'Web Development',
    'Analyst', 'Business Intelligence Management', 'Database', 'AMEF', 'Typography Design', 'Human Resources', 'Web Administration',
     'Motion Graphics', 'Content Creation', 'Communication Sciences', 'Web Writing', 'Web Scraping', 'Texture Mapping', 'Digital Specialist',
     'Offline Marketing', 'REST API Development', 'Market Research', 'Process Improvement', 'Digital Design', 'Writing', 'Monitoring',
     'Lean Manufacturing', 'SAP', 'React', 'Performance Analyst', 'Scrum Master', 'Team Coordination', 'Photography', 'Accounting Systems',
    'Style Correction', 'PMP Guidelines', 'Decision-Making Skills', 'Project Manager', 'Call Center Sales', 'Coaching', 'E-commerce',
    'Image Editing', 'Bitcoin', 'Public Relations', 'Python', 'Team Management Skills', 'Content for Social Media', 'Ticket Handling',
    'Digital Content Creation', 'Community Manager', 'Digital Media', 'Cloud Services', 'Windows Forms', 'Communication',
    'CRM (Customer Relationship Management)', 'Digital Advertising Sales', 'Web Design', 'Strategic Planning', 'English',
     'Literary Content Creation', '.NET Programming', 'UX UI Design', 'Keyword Research', 'Negotiation Techniques Management',
     'HTML Programming', 'Requirements Analysis', 'Social Media Marketing', 'ETL (Extract, Transform, Load)', 'Contouring',
    'Advertising', 'Google Ads', 'Editorial Design', 'Debugging', 'Digital Campaigns', 'Apache Spark', 'Software as a Service (SaaS)',
     'Java Programming', 'Responsive Design', 'Business Development', 'IT Solutions Manager', 'Postman', 'Azure DevOps', 'Adobe Photoshop',
     'Canva', 'Administrative Skills', 'DevOps', 'Angular', 'Version Control', 'Front-End Developer', 'Facebook Ads Campaign Implementation',
     'Virtual Assistance', 'Web Roles', 'Administration', 'Marketing Strategies', 'Fullstack Developer', 'API Consumption',
    'Supplier Relations', 'Computer Maintenance', 'Project Management', 'Voiceover', 'Advertising Design', 'Project Coordination',
    'Sales', 'Backend Development', 'User Flow Design', 'Interpersonal Communication Skills', 'Call Center Administration',
     'Email Marketing', 'MySQL', 'Good Spelling', 'Digital Campaign Implementation', 'Brand Design', 'Interior Architecture',
    'UI Design', 'Graphic Design', 'Consulting', 'ETL Process', 'React Hooks']

tools = ['Instagram', 'Zend Framework', 'Actionscript', 'Asana', 'Nats', 'Magento', 'Sprinklr', 'Outlook',
         'Mailchimp', 'HubSpot', 'ClickUp', 'Microsoft SQL', 'Twitter Analytics', 'WordPress', 'Social Media',
 'AWS', 'Monday', 'Adobe Audition', 'Correction', 'PowerPoint', 'HTML', 'Adobe Illustrator', 'Jira', 'Socialbakers',
 'Vue.js', 'Staging', 'Project', 'Scrum Methodology', 'Spring', 'Social Ads', 'Figma', 'Google Tag Manager', 'C#',
'Web Filtering', 'Solidworks', 'Spark', 'Google Cloud', 'TypeScript', 'Express', 'MacOS', 'Adobe', 'MongoDB', 'Microsoft Office',
'Excel', 'Facebook Business Manager', 'Next.js', 'tRPC', 'Metricool', 'Node.js', 'Office Suite', 'Salesforce', 'Marketing Foundations',
'Brandwatch', 'Facebook Analytics', 'Access', 'Sketchup', 'Facebook', 'Google', 'Looker Studio', 'SQL', 'Flutter', 'Django', 'JavaScript',
 'Supermetrics', 'Adwords / Google Ads', 'Adobe InDesign', 'Adobe After Effects', 'Writing', 'Final Cut', 'Adobe Creative Cloud',
'React Native', 'DaVinci Resolve', 'macOS', 'Microsoft Project', 'Linux', 'SAP', 'React', 'Google Docs', 'SEO', 'Java', 'Sysomos',
 'Trello', 'Google Marketing Platform', 'Cashier', 'Linux Commands', 'GraphQL', 'Python', 'Blender', 'Facebook Insights',
'Adobe Lightroom', 'Adobe Experience', 'Xcode', 'G Suite', 'Fanpage Karma', 'Word', 'Miro', 'Shopify', 'Marketplace', 'Trady',
'YouTube', 'Adobe Suite', 'Autocad', 'Advisor', 'TalkWalker', 'Less', 'VSCode', 'PostgreSQL', 'Bootstrap', 'Emplifi', 'Sprout Social',
'Programmatic Advertising', 'Golang', 'Social Media Design', 'Adobe Premiere', 'Git', 'Visual Studio', 'LinkedIn Ads', 'Power BI',
 'Microsoft Azure', 'Material UI', 'Google Data Studio', 'CSS', 'Cinema 4D', 'Windows', 'Postman', 'Jupiter X', 'Adobe Photoshop',
'Tableau', 'Meltwater', 'Canva', 'Visual Basic for Applications', 'Docker', 'Facebook Business', 'Google Workspace', 'Cake PHP', 'Hootsuite',
'Webex', 'Facebook Ads', 'Zoho', 'CRM', 'Odoo', 'Google Analytics', 'Tailwind', 'Cisco', 'GitHub', 'LinkedIn', 'Microsoft Teams',
'Oracle SQL', 'Basecamp', 'Smart Contracts', 'Keynote', 'MySQL', 'Angular.js', 'BMC Remedy']


aptitudes = ['Service Attitude',
'Adaptable',
'Kind',
'Analytical',
'Passionate',
'Passionate About Technology',
'Attention to Detail',
'Autodidact',
'Autonomy in Work',
'Good Relationship',
'Good Attitude',
'Business Management',
'Search for Excellence (Initiative)',
'Adaptability',
'Committed',
'Commitment',
'Communication',
'Assertive Communication',
'Effective Communication',
'Efficient Communication',
'With Initiative',
'Reliable',
'Consistent',
'Content Creation and Editing',
'Creative',
'Dedicated',
'Mental Dexterity',
'Dynamic',
'Disciplined',
'Willing to Teach',
'Willing to Learn',
'Efficient',
'Empathy',
'Entrepreneurial',
'Empathetic',
'Enthusiastic',
'Excellent Presentation',
'Word Fluency',
'Flexible',
'Team Management',
'Team Management',
'Sales Enthusiast',
'Honest',
'Emotionally Intelligent',
'Leadership',
'Leadership and Team Management',
'Time Management',
'Multidisciplinary',
'Negotiation',
'Organized',
'Organized',
'Goal-Oriented',
'Results-Oriented',
'Project Owner',
'Patient',
'Creative Thinking',
'Critical Thinking',
'Logical Thinking',
'Strategic Thinking',
'Perfectionist',
'Persistent',
'Pragmatic',
'Pragmatic',
'Proactive',
'Productive',
'Punctual',
'Resilient',
'Problem Solving',
'Responsible',
'Quick Learner',
'Sense of Urgency',
'Sociable',
'Problem Solving',
'Tenacious',
'Work Under Pressure',
'Customer-oriented',
'Teamwork',
'Teamwork',
'Versatile',
'Effective Communication',
'Creative']


languages = [
    'Spanish',
    'English',
    'Chinese Mandarin',
    'Hindi',
    'Arabic',
    'Portuguese',
    'Bengali',
    'Russian',
    'French',
    'German',
    'Japanese',
    'Turkish',
    'Chinese Cantonese',
    'Italian',
    'Swedish',
    'Indonesian',
    'Vietnamese',
    'Korean',
    'Polish',
    'Ukrainian',
    'Maltese',
    'Czech',
    'Hungarian',
    'Dutch',
    'Greek',
    'Swedish',
    'Norwegian',
    'Danish',
    'Finnish',
    'Egyptian Arabic',
    'Hebrew',
    'Thai',
    'Malay',
    'Filipino',
    'Afrikaans',
    'Luganda'
]

In [142]:
def encontrar_skills(Req_txt):
    coincidencias = [skill for skill in skills if re.search(r'\b{}\b'.format(re.escape(skill.lower())), Req_txt.lower())]
    return coincidencias

def lista_a_diccionarios(skills_1):
    return [
        {
            'name': skill,
            'level': 'BASIC',
            'experience': 0
        }
        for skill in skills_1
    ]

In [143]:
def encontrar_tools(Req_txt):
    coincidencias = [tool for tool in tools if re.search(r'\b{}\b'.format(re.escape(tool.lower())), Req_txt.lower())]
    return coincidencias

def diccionarios_2(tools_1):
    return [
        {
            'name': tool,
            'level': 'BASIC',
            'experience': 0
        }
        for tool in tools_1
    ]

In [144]:
def encontrar_aptitudes (Req_txt):
    coincidencias = [aptitud for aptitud in aptitudes if re.search(r'\b{}\b'.format(re.escape(aptitud.lower())), Req_txt.lower())]
    return coincidencias

def diccionarios_3(aptitudes_1):
    return [
        {
            'name': Aptitud

        }
        for Aptitud in aptitudes_1
    ]

In [145]:
def encontrar_languages (Req_txt):
    coincidencias = [language for language in languages if re.search(r'\b{}\b'.format(re.escape(language.lower())), Req_txt.lower())]
    return coincidencias

def diccionarios_4(languages_1):
    return [
        {
            'name': language,
            'level': 'BASIC'
        }
        for language in languages_1
    ]

In [146]:
def Encontrar_info(dataframe):

    dataframe['skills'] = dataframe['Req_txt'].apply(encontrar_skills)

    dataframe['skills_1'] = dataframe['Req_txt'].apply(encontrar_skills)
    dataframe = dataframe.drop('skills_1', axis=1)


    dataframe['tools_1'] = dataframe['Req_txt'].apply(encontrar_tools)

    dataframe['tools'] = dataframe['tools_1'].apply(diccionarios_2)
    dataframe = dataframe.drop('tools_1', axis=1)


    dataframe['aptitudes_1'] = dataframe['Req_txt'].apply(encontrar_aptitudes)

    dataframe['aptitudes'] = dataframe['aptitudes_1'].apply(diccionarios_3)
    dataframe = dataframe.drop('aptitudes_1', axis=1)


    dataframe['languages_1'] = dataframe['Req_txt'].apply(encontrar_languages)

    dataframe['languages'] = dataframe['languages_1'].apply(diccionarios_4)
    dataframe = dataframe.drop('languages_1', axis=1)

    dataframe['languages'] = dataframe['languages'].replace(' ', np.nan)
    return dataframe
df = Encontrar_info(df)

In [170]:
def Transform_df (dataframe):
    new_columns = ['id','companyId','createdAt','availableSlots','scholarity','workhours','locationConditions','nationalRemote', 'minSalary','maxSalary','minAge', 'maxAge','sex','yearsOfExperience','status','updatedAt', 'driversLicense', 'degree','validPassport','validVisa','nationalRelocation','internationalRelocation','availabilityToTravel','seniority','showSalaryRange','state','postalCode','slug','latitude','longitude','companyName','companyImg','originalDate','country']

    dataframe = dataframe.assign(**{col: np.nan for col in new_columns})
    dataframe = dataframe.rename(columns={'We offer': 'benefits'}) 
    dataframe = dataframe.rename(columns={'locacion': 'city'})
    dataframe = dataframe.rename(columns={'resumen': 'description'})
    dataframe = dataframe.rename(columns={'url': 'externalLink'})

    order_of_columns = ['id','companyId','name','description','createdAt','availableSlots','skills','aptitudes','tools','languages','benefits','scholarity','workhours','locationConditions','nationalRemote', 'minSalary','maxSalary','minAge', 'maxAge','sex','yearsOfExperience','status','country','updatedAt', 'driversLicense', 'degree','validPassport','validVisa','nationalRelocation','internationalRelocation','availabilityToTravel','seniority','showSalaryRange','state','city','postalCode','slug','latitude','longitude','companyName','companyImg', 'externalLink','originalDate']
    dataframe = dataframe[order_of_columns]

    dataframe['country'].fillna('Mexico', inplace = True)
    dataframe['sex'].fillna('OTHER', inplace = True)
    dataframe['companyId'].fillna('epam', inplace = True)
    dataframe['status'].fillna('SUBMITED', inplace = True)
    dataframe['minAge'].fillna(18, inplace = True)
    return dataframe

In [171]:
df = Transform_df (df)

In [172]:
def generate_slug(row):
    """
    Concatenar companyId, name y el resultado de nanoid
    """
    if row["companyId"] is not None and row["name"] is not None:
        return slugify(" ".join([row["companyId"], row["name"], generate(size=10)]))
    return None

In [173]:
df["slug"] = df.apply(generate_slug, axis=1)

In [174]:
def translate_deep_translator(text, target_language="es"):
    if isinstance(text, list):
        return [translate_deep_translator(item, target_language) for item in text]
    elif isinstance(text, dict):
        return {
            key: translate_deep_translator(value, target_language)
            for key, value in text.items()
        }
    elif isinstance(text, str):
        translated_text = GoogleTranslator(
            source="auto", target=target_language
        ).translate(text)
        return translated_text
    else:
        return text

def translate_columns_deep_translator(
    df_a_traducir, column_names, target_language="es"
):
    for column_name in column_names:
        df_a_traducir[column_name] = df_a_traducir[column_name].apply(
            lambda x: translate_deep_translator(x, target_language)
        )
    return df_a_traducir

In [175]:
columnas_a_traducir = ['description', 'skills', 'tools', 'aptitudes', 'languages', 'benefits']
df = translate_columns_deep_translator(df, columnas_a_traducir)

In [176]:
df

,id,companyId,name,description,createdAt,availableSlots,skills,aptitudes,tools,languages,...,state,city,postalCode,slug,latitude,longitude,companyName,companyImg,externalLink,originalDate
0,NaN,epam,\n Application Support (Mobile)\n ...,¿Es usted un especialista en soporte de aplica...,NaN,NaN,"[Infraestructura, MicrosoftSQL, Reaccionar, Pi...",[{'name': 'Flexible'}],"[{'name': 'MicrosoftSQL', 'level': 'BÁSICO', '...","[{'name': 'Inglés', 'level': 'BÁSICO'}]",...,NaN,Guadalajara,NaN,epam-application-support-mobile-guadalajara-me...,NaN,NaN,NaN,NaN,[https://www.epam.com/careers/job-listings/job...,NaN
1,NaN,epam,\n Big Data Support Engineer\n \...,EPAM es un proveedor líder mundial de servicio...,NaN,NaN,"[Infraestructura, Programación, Pitón, Manejo ...",[{'name': 'Flexible'}],"[{'name': 'Acceso', 'level': 'BÁSICO', 'experi...","[{'name': 'Inglés', 'level': 'BÁSICO'}]",...,NaN,Guadalajara,NaN,epam-big-data-support-engineer-guadalajara-mex...,NaN,NaN,NaN,NaN,[https://www.epam.com/careers/job-listings/job...,NaN
2,NaN,epam,\n Cloud Operations (Linux)\n \n...,DESCRIPCIÓN ¿Es usted un profesional experimen...,NaN,NaN,"[Infraestructura, Servidor de windows, Pitón, ...",[{'name': 'Flexible'}],"[{'name': 'Acceso', 'level': 'BÁSICO', 'experi...","[{'name': 'Inglés', 'level': 'BÁSICO'}]",...,NaN,Guadalajara,NaN,epam-cloud-operations-linux-guadalajara-mexico...,NaN,NaN,NaN,NaN,[https://www.epam.com/careers/job-listings/job...,NaN
3,NaN,epam,\n Data Consultant / Data Science Engin...,DESCRIPCIÓNBuscamos un consultor de datos/inge...,NaN,NaN,"[Análisis de datos, Análisis de los datos, Pit...","[{'name': 'Comunicación'}, {'name': 'Flexible'}]","[{'name': 'AWS', 'level': 'BÁSICO', 'experienc...","[{'name': 'Inglés', 'level': 'BÁSICO'}]",...,NaN,Mexico City,NaN,epam-data-consultant-data-science-engineer-mex...,NaN,NaN,NaN,NaN,[https://www.epam.com/careers/job-listings/job...,NaN
4,NaN,epam,\n Data Integration Engineer\n \...,DESCRIPCIÓN ¡Únase a nuestro dinámico equipo c...,NaN,NaN,"[Análisis de los datos, Inglés]","[{'name': 'Eficiente'}, {'name': 'Flexible'}]","[{'name': 'Proyecto', 'level': 'BÁSICO', 'expe...","[{'name': 'Inglés', 'level': 'BÁSICO'}]",...,NaN,Mexico City,NaN,epam-data-integration-engineer-mexico-city-mex...,NaN,NaN,NaN,NaN,[https://www.epam.com/careers/job-listings/job...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,NaN,epam,\n Sr. Salesforce Developer\n \n...,EPAM es un proveedor líder mundial de servicio...,NaN,NaN,"[Sí, Habilidades de comunicación, Nodo.js, Esc...","[{'name': 'Comunicación'}, {'name': 'Flexible'}]","[{'name': 'HTML', 'level': 'BÁSICO', 'experien...","[{'name': 'Inglés', 'level': 'BÁSICO'}]",...,NaN,,NaN,epam-sr-salesforce-developer-remote-kuhl5fumut,NaN,NaN,NaN,NaN,[https://www.epam.com/careers/job-listings/job...,NaN
101,NaN,epam,\n Sr. iOS Developer\n \n ...,EPAM es un proveedor líder mundial de servicio...,NaN,NaN,"[Infraestructura, Escribiendo, Comunicación, I...","[{'name': 'Comunicación'}, {'name': 'Flexible'}]","[{'name': 'Acceso', 'level': 'BÁSICO', 'experi...","[{'name': 'Inglés', 'level': 'BÁSICO'}]",...,NaN,,NaN,epam-sr-ios-developer-remote-uwsebnnlwo,NaN,NaN,NaN,NaN,[https://www.epam.com/careers/job-listings/job...,NaN
102,NaN,epam,\n Testing/QA Manager - Microsoft Dynam...,DESCRIPCIÓN ¿Es usted un administrador de prue...,NaN,NaN,[Inglés],[{'name': 'Flexible'}],"[{'name': 'Acceso', 'level': 'BÁSICO', 'experi...","[{'name': 'Inglés', 'level': 'BÁSICO'}]",...,NaN,Remote\n \n \n ...,NaN,epam-testing-qa-manager-microsoft-dynamics-365...,NaN,NaN,NaN,NaN,[https://www.epam.com/careers/job-listings/job...,NaN
103,NaN,epam,\n UX Content Designer / UX Writer\n ...,DESCRIPCIÓN Únase a nosotros como diseñador de...,NaN,NaN,"[figura, Habilidades de comunicación, Gestión ...","[{'name': 'Comunicación'}, {'name': 'Flexible'}]","[{'name': 'Proyecto', 'level': 'BÁSICO', 'expe...","[{'name': 'Inglés', 'level': 'BÁSICO'}]",...,NaN,,NaN,epam-ux-content-design